In [2]:
import os
import sys
import time

import GCode
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Draw a Shapes

Attempt at programmatically drawing shapes.

All units in mm. ```1``` = ```1 mm```.

In [3]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.status


'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

In [23]:
radii = list(range(1,21))

In [24]:
radii

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [25]:
X0=np.cumsum(radii)

In [26]:
X0

array([  1,   3,   6,  10,  15,  21,  28,  36,  45,  55,  66,  78,  91,
       105, 120, 136, 153, 171, 190, 210])

In [27]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


In [28]:
def circle(center_x=10, center_y=10, radius=10, laser_pwm=255, clockwise=True):
    prog = GCode.GCode()
    prog.G0(X=(center_x - radius), Y=(center_y))
    prog.M3(S=laser_pwm)
    arc_cfg={
        "X": (center_x - radius),
        "Y": (center_y),
        "I": radius,
        "J": 0,      
    }
    if clockwise:
        prog.G2(**arc_cfg)
    else:
        prog.G3(**arc_cfg)
    prog.M5()
    return prog


In [29]:
prog=GCode.GCode()
prog+=init()
for x0, radius in zip(X0, radii):
    prog+=circle(center_x=x0, center_y=radius, radius=radius, laser_pwm=255, clockwise=True)
        
prog+=end()

In [30]:
prog.machine=cnc

prog.run()